In [2]:
import os  # interact with the filesystem
from subprocess import Popen, PIPE, STDOUT  # enable calling commandline
import matplotlib.pyplot as plt  # manipulate figures
import seaborn as sns  # display results
import pandas as pd   # manipulate tabular 

In [5]:
data_dir = '/projects/niblab/bids_projects/Experiments/Bevel/testing_beta'
print('Our working directory: {}'.format(data_dir))

Our working directory: /Users/nikkibytes/Documents/niblunc/testing_beta


In [6]:
events_file = '/projects/niblab/bids_projects/Experiments/Bevel/testing_beta/sub-001/func/sub-001_task-prob_run-1_events.tsv'
events_df = pd.read_csv(events_file, sep='\t', na_values="n/a")

print(events_df.head())

   Unnamed: 0  ID    run     onset  duration img_pair choice  side trial_type  \
0           0   1  run01  108.8459       5.0       EF      F     1     reward   
1           1   1  run01  123.8299       5.0       CD      D     2     punish   
2           2   1  run01  139.8476       5.0       CD      D     2     punish   
3           3   1  run01  153.8315       NaN       CD   Miss  Miss       Miss   
4           4   1  run01  166.8329       5.0       AB      A     2     reward   

      matched response_time reinforcer_response  
0  mismatched      1.569796                 NaN  
1     matched      1.601124                 NaN  
2     matched      1.600702      explore_punish  
3        Miss          Miss       miss_response  
4     matched      1.580793                 NaN  


In [7]:
atlas_txt = os.path.join(data_dir,
                         "derivatives",
                         "data",
                         "Schaefer2018_100Parcels_7Networks_order.txt")
atlas_df = pd.read_csv(atlas_txt, sep="\t", header=None)
print(atlas_df.head())

atlas_df.drop([2, 3, 4, 5], axis='columns', inplace=True)
print(atlas_df.head())

atlas_df.rename({0: 'index', 1: 'regions'}, axis='columns', inplace=True)
print(atlas_df.head())


atlas_df.replace(regex={'7Networks_(.*)': '\\1'}, inplace=True)
print(atlas_df.head())

atlas_tsv = atlas_txt.replace(".txt", ".tsv")
atlas_df.to_csv(atlas_tsv, sep="\t", index=False)


   0                   1    2   3    4  5
0  1  7Networks_LH_Vis_1  120  18  131  0
1  2  7Networks_LH_Vis_2  120  18  132  0
2  3  7Networks_LH_Vis_3  120  18  133  0
3  4  7Networks_LH_Vis_4  120  18  135  0
4  5  7Networks_LH_Vis_5  120  18  136  0
   0                   1
0  1  7Networks_LH_Vis_1
1  2  7Networks_LH_Vis_2
2  3  7Networks_LH_Vis_3
3  4  7Networks_LH_Vis_4
4  5  7Networks_LH_Vis_5
   index             regions
0      1  7Networks_LH_Vis_1
1      2  7Networks_LH_Vis_2
2      3  7Networks_LH_Vis_3
3      4  7Networks_LH_Vis_4
4      5  7Networks_LH_Vis_5
   index   regions
0      1  LH_Vis_1
1      2  LH_Vis_2
2      3  LH_Vis_3
3      4  LH_Vis_4
4      5  LH_Vis_5


In [8]:
out_dir = os.path.join(data_dir, "output")
work_dir = os.path.join(data_dir, "work")
atlas_mni_file = os.path.join(data_dir,
                              "derivatives",
                              "data",
                              "Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz")
cmd = """\
nibs -c WhiteMatter CSF \
-sp MNI152NLin2009cAsym \
-w {work_dir} \
-a {atlas_mni_file} \
-l {atlas_tsv} \
{bids_dir} \
fmriprep \
{out_dir} \
participant 
""".format(atlas_mni_file=atlas_mni_file,
           atlas_tsv=atlas_tsv,
           bids_dir=os.path.join(data_dir),
           out_dir=out_dir,
           work_dir=work_dir)

# Since we cannot run bash commands inside this tutorial
# we are printing the actual bash command so you can see it
# in the output
print("The Example Command:\n", cmd)

# call nibs
p = Popen(cmd, shell=True, stdout=PIPE, stderr=STDOUT)

while True:
    line = p.stdout.readline()
    if not line:
        break
    print(line)

The Example Command:
 nibs -c WhiteMatter CSF -w /Users/nikkibytes/Documents/niblunc/testing_beta/work -a /Users/nikkibytes/Documents/niblunc/testing_beta/derivatives/data/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz -l /Users/nikkibytes/Documents/niblunc/testing_beta/derivatives/data/Schaefer2018_100Parcels_7Networks_order.tsv /Users/nikkibytes/Documents/niblunc/testing_beta fmriprep /Users/nikkibytes/Documents/niblunc/testing_beta/output participant

b'190724-17:16:21,756 nipype.workflow INFO:\n'
b"\t Workflow nibetaseries_participant_wf settings: ['check', 'execution', 'logging', 'monitoring']\n"
b'190724-17:16:21,772 nipype.workflow INFO:\n'
b'\t Running in parallel.\n'
b'/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.\n'
b'  from ._conv import register_converte

In [ ]:
corr_mat_path = os.path.join(out_dir, "NiBetaSeries", "nibetaseries", "sub-001", "func")
trial_types = ['punish','reward']
filename_template = "sub-001_task-prob_run-1_bold_space-MNI152NLin2009cAsym_preproc_trialtype-{trial_type}_matrix.tsv"
pd_dict = {}
for trial_type in trial_types:
    file_path = os.path.join(corr_mat_path, filename_template.format(trial_type=trial_type))
    pd_dict[trial_type] = pd.read_csv(file_path, sep='\t', na_values="n/a", index_col=0)
# display example matrix
print(pd_dict[trial_type].head())

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True, figsize=(10, 30),
                         gridspec_kw={'wspace': 0.025, 'hspace': 0.075})

cbar_ax = fig.add_axes([.91, .3, .03, .4])
r = 0
for trial_type, df in pd_dict.items():
    g = sns.heatmap(df, ax=axes[r], vmin=-.5, vmax=1., square=True,
                    cbar=True, cbar_ax=cbar_ax)
    axes[r].set_title(trial_type)
    # iterate over rows
    r += 1
plt.tight_layout()